#### 生成 Title View需要的数据
暂定群体是all，直接使用titles_all.csv数据


In [40]:
import pandas as pd
import json
from tqdm import tqdm
import random
import math
import numpy as np

titles_all_df = pd.read_csv('题目数据特征/titles_all.csv')
titleInfo_df = pd.read_csv("../dataset/Data_TitleInfo.csv")
st_df = pd.read_csv("人题数据/stu_title.csv")
mergeData = pd.read_csv('../dataset/Data_MergedNew.csv')

In [45]:

subs = sorted(titleInfo_df['sub_knowledge'].unique())
kls = sorted(titleInfo_df['knowledge'].unique())

OutputArr = []
for kl in tqdm(kls):
    dataobj = {"knowledge":kl}
    tmpsubs = []
    for sub in subs:
        if sub[:len(kl)] == kl:
            tmpsubs.append(sub)
    dataobj["sub_knowledges"] = tmpsubs
    
    List =  titleInfo_df[titleInfo_df['knowledge']==kl]['title_ID'].unique().tolist()
    dataobj["titleNum"] = len(List)
    
    # 提交次数和得分表现
    tmplist = []
    for title in List:
        titleObj = {}
        titleObj["t_id"] = title
        
        tdf = titles_all_df[titles_all_df['title_ID']==title]
        titleObj["belong"] = tdf["sub_knowledge"].tolist()[0]
        titleObj["t_mean_sbmnum"] = tdf['t_mean_sbmnum'].tolist()[0]
        titleObj["t_mean_score"] = tdf['t_mean_score'].tolist()[0]
        
        # 题目的所有提交时间
        titleObj["SubmitArr"] = mergeData[mergeData['title_ID']==title]['time'].to_list()
        titleObj["SubmitNum"] = len(titleObj["SubmitArr"])
        titdf = st_df[st_df['title_ID']==title]
        fulscore = titdf['score'].tolist()[0]
        titleObj["t_sbmNum"] = titdf.shape[0]
        titleObj["t_Fullscore"] = fulscore
        sc = titdf['st_score'].value_counts().to_dict()
        newobj = {}
        for key, value in sc.items():
            k = key * fulscore
            newobj[k] = value/titdf.shape[0]
        newobj = {k:newobj[k] for k in sorted(newobj)}
        
        obj ={"a":[], "b":[], "c":[]}
        v = 0
        for key, value in newobj.items():
            obj['a'].append(key)
            obj['b'].append(value)
            obj['c'].append(v)
            v = v+ value
        # print(obj)
        titleObj["t_score_distribute"] = obj
        
        titledf = mergeData[mergeData['title_ID']==title]
        titledf = titledf[titledf['state'].isin(['Partially_Correct','Absolutely_Correct'])]
        
        titleObj["t_memArr"] = titledf['memory'].tolist()
        titleObj["t_tcArr"] = titledf['timeconsume'].tolist()
        
        tmplist.append(titleObj)
    dataobj["titles"] = tmplist
    
    OutputArr.append(dataobj)

json_str = json.dumps(OutputArr, ensure_ascii=False)
with open("TitleViewData.json", "w", encoding="utf-8") as file:
    file.write(json_str)
    

100%|██████████| 8/8 [00:01<00:00,  7.53it/s]


找各个指标的最大最小值帮助前端比例尺映射

In [46]:
min_pfm_tc = titles_all_df['t_min_pfm_tc'].min()
max_pfm_tc = titles_all_df['t_max_pfm_tc'].max()
print('时间复杂度范围',min_pfm_tc, max_pfm_tc)

min_pfm_mem = titles_all_df['t_min_pfm_mem'].min()
max_pfm_mem = titles_all_df['t_max_pfm_mem'].max()
print('空间复杂度范围',min_pfm_mem, max_pfm_mem)

min_mean_score = titles_all_df['t_mean_score'].min()
max_mean_score = titles_all_df['t_mean_score'].max()
print('平均分范围',min_mean_score, max_mean_score)
min_mean_sbmnum = titles_all_df['t_mean_sbmnum'].min()
max_mean_sbmnum = titles_all_df['t_mean_sbmnum'].max()
print('平均提交次数范围',min_mean_sbmnum, max_mean_sbmnum)

print('提交时间',mergeData['time'].min(), mergeData['time'].max())
valid_df = mergeData[mergeData['state'].isin(['Partially_Correct','Absolutely_Correct'])]
print('有效提交中')
print('最小mem',valid_df['memory'].min(),'最大mem', valid_df['memory'].max())
print('最小tc',valid_df['timeconsume'].min(),'最大tc', valid_df['timeconsume'].max())
# 真服了，就是又65536的mem，还是PC的提交

时间复杂度范围 1 400
空间复杂度范围 128 65536
平均分范围 13.601599165652702 49.81823577416723
平均提交次数范围 2.4038004750593824 8.848640705363703
提交时间 1693471583.0 1706158726.0
有效提交中
最小mem 160 最大mem 65536
最小tc 2 最大tc 400
